In [3]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import numpy as np
import pandas as pd
import math

What is the optimal set up for Alice’s new pool? Think about how you’d approach it and explain why.

Install weights = 0.5 и 0.5 and install spot price = market price. The pool wil have [13783, 5100000]. And will be rest of Tokens A = 43216

In [10]:
#pool V=x^wx*y^wy
rest_C=43216 # rest of Tokens A
pool = [13783, 510000]
fee=0.01
weight = [0.5, 0.5]
price=[37, 1]
price_new=[28, 1]
inv = pool[0]**weight[0]*pool[1]**weight[1]
print('inv= ', inv)
#inv=  29937260.646224797

inv=  83841.09970652818


Due to a huge price dump, the price of token C on external markets drops to 28 token D. Now, Arbitrage traders get to work. Image Alice wanted to re-balance the pool herself, how much token C would she need in order to re-balance the pool in one trade? Keep in mind swap fees and slippage! - 
Make function for calculation - calc_out_from_in (calc_out_given_in)

In [11]:
pool = [13783, 510000]
fee=0.01
weight = [0.5, 0.5]

 #function for computer spot price
def sp(pool, weight): #function for computer spot price
    return pool[1]*weight[0]/pool[0]/weight[1]
#print(sp(pool, weight)) # for check

def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step


In [15]:
def calc_out_given_in(pool, weight, fee, amount, n):
    if n==0:
        out = amount*(1-pool[0]/(pool[0]+pool[1]*(1-fee))**(weight[0]/weight[1]))
    if n==1:
        out = amount*(1-pool[1]/(pool[1]+pool[0]*(1-fee))**(weight[1]/weight[0]))    
    return out  

#--calculate the amount tokens for trade-----
def amount(pool, price_new,  weight, fee, step):
    
    a=1
    if (pool[1]/pool[0])>(price_new[0]/price_new[0]):
        for i in range(1, 13000, step):
            #print('i= ', i)
            if ((pool[0]+i)*price_new[0]/price_new[1])>(pool[1]+calc_out_given_in(pool, weight, fee, 1, 0)):
                #print('решение= ', i)
                if a==1:
                    a=i
    return a

print('amount= ', amount(pool, price_new, weight, fee, 1))

amount=  4432


Due to a huge price dump, the price of token C on external markets drops to 28 token D. Now, Arbitrage traders get to work. Image Alice wanted to re-balance the pool herself, how much token C would she need in order to re-balance the pool in one trade? Keep in mind swap fees and slippage!

Check that this calculation is right making swap

In [16]:
#function for swap
def swap(pool1, weight1,amount1,fee1, n):
    pool2=[0,0]
    if n==0:
        b = amount1*(1-pool1[0]/(pool1[0]+pool1[1]*(1-fee1))**(weight1[0]/weight[1]))
        pool2[0]=pool1[0]+amount1
        pool2[1]=pool1[1]-b
    if n==1:
        b = amount1*(1-pool1[1]/(pool1[1]+pool1[0]*(1-fee1))**(weight1[1]/weight[0]))
        pool2[1]=pool1[1]+amount1
        pool2[0]=pool1[0]-b    
    return pool2

pool3=swap(pool, weight, 4432, fee, 0)
# make swap
print('pool = ', pool3 )
print('spot price become = ',sp(pool3,weight))


pool =  [18215, 505685.77184908703]
spot price become =  27.762051707333903


3.Imagine Alice’s pool would be twice as big. How much liquidity would she need for the same price change?

In [17]:
pool4 = [13783*2, 510000*2]
print('amount = ',amount(pool4, price_new, weight, fee, 1))
#new amount equal 8863. Notice that this amount twice

amount =  8863


Explore the general relation between the size of the pool (liquidity m in token C and n in token D) and price changes with a series of experiments.
• derive an expression
• plot a graph


In [18]:
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
#-----------
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump
#------------------

#Plot curves of the pool=[37, 1] and [370, 10] 
pool1=[37, 1]
pool2=[370, 10]

def inv(pool, weight):
    return pool[0]**weight[0]*(pool[1]**weight[1])
#--------------
inv1=inv(pool1, weight)
inv2=inv(pool2, weight)
#-------------
def f(x,weight, inv):
    return (inv/(x**weight[0]))**(1/weight[1]) 

x1 = np.arange(0.1, 10, 0.01)

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=x1, y=(f(x1,[0.5, 0.5],inv1)),line=dict(color='green'), name="Pool1"), secondary_y=False)
fig.add_trace(go.Scatter(x=x1, y=(f(x1,[0.5,0.5],inv2)), line=dict(color='red'), name="Pool2"), secondary_y=False)
fig.update_layout(title_text='<b>Curves of Pool1 and Pool2</b>')
fig.update_xaxes(title_text='')
fig.update_yaxes(range=[0, 3000])
fig.update_xaxes(range=[-0.1, 10])

fig.show()


The curves show that the greater the liquidity, the smaller the slipperage